In [3]:
# Normal import
import numpy as np
from collections import defaultdict
import random
from numpy import random
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from dwave.system import LeapHybridSampler
import matplotlib



## Quantum Part
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod
from dwave.system import LeapHybridSampler
from dwave.system import LeapHybridCQMSampler


#Page Size
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
def add_front(a,b):
    if len(a) < len(b):
        c = b.copy()
        c[:len(a)] += a
    else:
        c = a.copy()
        
        c[:len(b)] += b  
    return c

In [5]:
def add_end(a,b):
    if len(a) < len(b):
        c = b.copy()
        c[-len(a):] += a
    else:
        c = a.copy()
        c[-len(b):] += b  
    return c

In [6]:
def QUBO_Cons_ineq_normal(QUBO_pre, lhs, symbol, rhs, round_num = None, penalty = None, dictionary = None):
    #print("QUBO_Cons_ineq_normal")
    
    label = 0
    
    if penalty == None:
        
        penalty = 1e3
        label = 1
    
    #print("label >= ", label, ", penalty is",penalty)
    
    disturbance = 1e-6
    
    QUBO = QUBO_pre.copy()
    
    if symbol == "<=":
        
        #print("symbol ====> <=")
        
        min_value = np.sum(lhs[lhs<0])
        
        Slack_Var_Add_num = max(0,np.ceil(np.log2(rhs - min_value + disturbance)))
        
        # Coeff = C for C * s
        Slack_Var_Add_coeff = 2 ** np.arange(Slack_Var_Add_num)
        
        #print("Slack_Var_Add_num:", Slack_Var_Add_num)
        #print("Slack_Var_Add_coeff:", Slack_Var_Add_coeff)
        
        for slack_index in range(int(Slack_Var_Add_num)):
            
            if not dictionary == None:
                
                next_index = len(dictionary)
            
                dictionary["s_%d_%d"%(round_num, slack_index)] = next_index
            
            QUBO = np.pad(QUBO, [(0, 1), (0, 1)], mode='constant', constant_values = 0)   #local QUBO arithmaic
            

        var_matched = np.zeros(len(QUBO))
            
        #The Final answer of is  (∑Ax + ∑Cs - M)^2
        #  (∑Ax)*(∑Ax) + (∑Cs)*(∑Cs) + (∑Ax)(∑Cs) + (∑Cs)(∑Ax) 
        # - 2(∑MCs) - 2(∑MAx)
            
        #(Ax)
        var_matched_1 = add_front(lhs, var_matched)
        #print("var_matched_1:",var_matched_1)
        
        #(Cs)
        if Slack_Var_Add_num > 0:
            var_matched_2 = add_end(Slack_Var_Add_coeff, var_matched)        
            #print("var_matched_2:",var_matched_2)
        # M
        M = rhs
        
        
        #(∑Ax)*(∑Ax)
        QUBO += penalty * (np.outer(var_matched_1, var_matched_1))
        
        if Slack_Var_Add_num > 0:       
            
            #(∑Cs)*(∑Cs)   
            QUBO += penalty * (np.outer(var_matched_2, var_matched_2))

            #(∑Ax)(∑Cs) + (∑Cs)(∑Ax) 
            QUBO += penalty * (np.outer(var_matched_1, var_matched_2) + np.outer(var_matched_2, var_matched_1))

            # -2(∑MCs) 
            QUBO -= penalty * 2 * M *(np.diag(var_matched_2))
            
        #- 2(∑MAx)
        QUBO -= penalty * 2 * M *(np.diag(var_matched_1))        
              
    else: # symbol == ">=":
        
        #print("symbol ==> >=")
        
        max_value = np.sum(lhs[lhs>0])
        
        Slack_Var_Add_num = max(0,np.ceil(np.log2(max_value - rhs + disturbance)))
        
        Slack_Var_Add_coeff = 2 ** np.arange(Slack_Var_Add_num)
        
        #print("Slack_Var_Add_num:", Slack_Var_Add_num)
        #print("Slack_Var_Add_coeff:", Slack_Var_Add_coeff)
        
        for slack_index in range(int(Slack_Var_Add_num)):
            
            if not dictionary == None:
                
                next_index = len(dictionary)
            
                dictionary["s_%d_%d"%(round_num, slack_index)] = next_index
            
            QUBO = np.pad(QUBO, [(0, 1), (0, 1)], mode='constant', constant_values = 0)   #local QUBO arithmaic      
            
        var_matched = np.zeros(len(QUBO))
            
        #The Final answer of is  (∑Ax - ∑Cs - M)^2
        #  (∑Ax)*(∑Ax) + (∑Cs)*(∑Cs) - (∑Ax)(∑Cs) - (∑Cs)(∑Ax) 
        # + 2(∑MCs) - 2(∑MAx)
            
        #(Ax)
        var_matched_1 = add_front(lhs, var_matched)
            
        #(Cs)
        if Slack_Var_Add_num > 0:
            var_matched_2 = add_end(Slack_Var_Add_coeff, var_matched)        
            #print("var_matched_2:",var_matched_2)
    
        # M
        M = rhs
        
        
        #(∑Ax)*(∑Ax)
        QUBO += penalty * (np.outer(var_matched_1, var_matched_1))
        if Slack_Var_Add_num > 0:    
            
            #(∑Cs)*(∑Cs)   
            QUBO += penalty * (np.outer(var_matched_2, var_matched_2))

            #-(∑Ax)(∑Cs) - (∑Cs)(∑Ax) 
            QUBO -= penalty * (np.outer(var_matched_1, var_matched_2) + np.outer(var_matched_2, var_matched_1))

            # +2(∑MCs) 
            QUBO += penalty * 2 * M *(np.diag(var_matched_2))
            
        #- 2(∑MAx)
        QUBO -= penalty * 2 * M *(np.diag(var_matched_1))        

    #if no dictionary is needed. just do QUBO, _ = function(*args) 
    return QUBO, dictionary

In [7]:
def QUBO_Cons_ineq_0(QUBO_pre, lhs, symbol, rhs, penalty = None, dictionary = None):
    
    label = 0
    
    #set up a default penalty just in case ... 
    if penalty == None:
        
        penalty = 1e3
        label = 1
    
    #print("label => ", label, ", penalty is",penalty)
    
    QUBO = QUBO_pre.copy()
    
    # Contraint QUBOlize : 
    # Cons = x-xy when x <= y 
    if symbol == "<=":  
        
        vector_x = lhs == 1
        vector_y = lhs == -1
        
        QUBO += penalty * (np.diag(vector_x) - np.outer(vector_x, vector_y))     
        
        
    # Cons = y-yx when x >= y
    else: # symbol == ">=":
        
        vector_x = lhs == -1
        vector_y = lhs == 1
        
        QUBO += penalty * (np.diag(vector_x) - np.outer(vector_x, vector_y))     
    
    #if no dictionary is needed. just do QUBO, _ = function(*args) 
    return QUBO, dictionary

In [8]:
def QUBO_Cons_leq_1(QUBO_pre, lhs, symbol, rhs, penalty = None, dictionary = None):
    
    label = 0
    
    #set up a default penalty just in case ... 
    if penalty == None:
        
        penalty = 1e3
        label = 1
    
    #print("label => ", label, ", penalty is",penalty)
    
    QUBO = QUBO_pre.copy()
    
    # Cons = Sum(xy)
    #        ───┬───
    #           │
    #  np.triu( np.outer(lhs, lhs), k = 1)
    #
    QUBO += penalty * (np.triu( np.outer(lhs, lhs), k = 1))     
    
    # if no dictionary is needed. just do QUBO, _ = function(*args) 
    return QUBO, dictionary

In [9]:
def QUBO_Cons_geq_1(QUBO_pre, lhs, symbol, rhs, penalty = None, dictionary = None):
    
    label = 0
    
    #set up a default penalty just in case ... 
    if penalty == None:
        
        penalty = 1e3
        label = 1
    
    #print("label => ", label, ", penalty is",penalty)
    
    QUBO = QUBO_pre.copy()
    
    # Cons = 1 (- x - y) +  xy
    #           ───┬───    ──┬─
    #           diag(arg)    │
    #                        │ 
    #          np.triu( np.outer(lhs, lhs), k = 1)
    #
    QUBO += penalty * ( np.triu( np.outer(lhs, lhs), k = 1) - np.diag(lhs) )     
    
    # if no dictionary is needed. just do QUBO, _ = function(*args) 
    return QUBO, dictionary

In [10]:
def QUBO_Cons_eq(QUBO_pre, lhs, rhs, penalty = None, dictionary = None):
    
    label = 0
    
    #set up a default penalty just in case ... 
    if penalty == None:
        
        penalty = 1e3
        label = 1
    
    #print("label => ", label, ", penalty is",penalty)
    
    QUBO = QUBO_pre.copy()
    
    # Let assume  Ax = b =====> lhs * x = rhs
    #
    # Cons =    x'A'Ax   -   2Abx + (b^2) constant neglect 
    #           ───┬───      ──┬─
    #      np.outer(lhs,lhs)   │
    #                          │ 
    #                 2 * rhs * np.diag(lhs)
    #
    QUBO += penalty * ( np.outer(lhs,lhs) - 2 * rhs * np.diag(lhs) )     
    
    # if no dictionary is needed. just do QUBO, _ = function(*args) 
    return QUBO, dictionary

In [11]:
# QUBO_Cons_selection blue print

# if symbol == "="
# function(QUBO_Cons_eq)
#
# if not (normalize?)     
#  rhs  other   1   0
#        ─┬    ─┬  ─┬  
#         │     │   │  
#         │     │   └┤ lhs Bin && np.sum(lhs) == 0 && np.sum(abs(lhs))== 2 ? ─┬┤yes: function(QUBO_Cons_ineq_0)
#         │     │                                                             │ 
#         │     │                                                             └┤No: function(QUBO_Cons_ineq_normal)
#         │     │
#         │     └────┤ lhs Bin && symbol == "<=" ? &&─┬────┤yes: function(QUBO_Cons_leq_1)
#         │                                           │                                               
#         │                                           └────┤No (symbol == ">=") if lhs Bin && np.sum(lhs)== 2   ─┬┤yes: function(QUBO_Cons_geq_1)
#         │                                                                                                      │ 
#         │                                                                                                      └┤No: function(QUBO_Cons_ineq_normal)
#         │
#         └──────────┤ function(QUBO_Cons_ineq_normal)



# 1. get rid of inactive line(constraints) every time after we sampled output 10 times

# 2. relax constraint?  5x + 6y <= 4   x,y Bin   
#                  ==> 1.25x + 1.5y <= 1  
#                  ==>  x + y <= 1 ?

In [12]:
def QUBO_Cons_selection(QUBO_pre, lhs, symbol, rhs, round_num = None, penalty = None, dictionary = None):
    
    #print("yep enter")
    #print(lhs, symbol, rhs)
    
    if symbol == "=":
        
        QUBO, dictionary = QUBO_Cons_eq(QUBO_pre, lhs, rhs, penalty, dictionary)
        
        #print("=condition")
        
    elif rhs == 0:
        
        if (np.sum(lhs) == 0) and (np.sum(abs(lhs))== 2):
            
            QUBO, dictionary = QUBO_Cons_ineq_0(QUBO_pre, lhs, symbol, rhs, penalty, dictionary)
            
            #print("ineq0 condition")
            
        else:
        
            QUBO, dictionary = QUBO_Cons_ineq_normal(QUBO_pre, lhs, symbol, rhs, round_num, penalty, dictionary)
            
            #print("ineq0 else condition")
        
    elif rhs == 1:
        
        if (np.array_equal(lhs, lhs.astype(bool))) and (symbol == "<=") :
          
            QUBO, dictionary = QUBO_Cons_leq_1(QUBO_pre, lhs, symbol, rhs, penalty, dictionary)
            
            #print("fx <=1 condition")
            
        elif (np.array_equal(lhs, lhs.astype(bool))) and (len(lhs) == 2) :
                
            QUBO, dictionary = QUBO_Cons_geq_1(QUBO_pre, lhs, symbol, rhs, penalty, dictionary)
            
            #print("fx >=1 condition")
        
        else:
            
            QUBO, dictionary = QUBO_Cons_ineq_normal(QUBO_pre, lhs, symbol, rhs, round_num, penalty, dictionary)
            
            #print("ineq1 else condition")
    
    else:
        
        QUBO, dictionary = QUBO_Cons_ineq_normal(QUBO_pre, lhs, symbol, rhs, round_num, penalty, dictionary)
        
        #print("normal else condition")
        
    return QUBO, dictionary
        
            

## Objective
$$
\min C^{\intercal} \cdot x + t 
$$

In [13]:
def solve_master_old(C, t, QUBO_pre, new_Constraint, new_Symbol, New_rhs, round_num = None, Penalty = None, _print = False, dictionary = None):
    
    # OBJ
    QUBO = QUBO_pre

    # Lastest Constraint
    
    symbol_set = ["<=",">="] 
    index = symbol_set.index(new_Symbol)
    #print(New_rhs, index)
    
    symbol = (symbol_set[::int(np.sign(New_rhs))])[index]
    
    #if np.sign(New_rhs) == -1:
    #    symbol = ">="
        
    #  np.sign(New_rhs) == -1 ==>  switch >= and <= to the other symbol
    #  np.sign(New_rhs) == 1 ==>  remain >= and <= 
    #
    
    cons_add_lhs_final = np.sign(New_rhs) * new_Constraint
    cons_add_rhs_final = np.sign(New_rhs) * New_rhs

    # print out the lasted the equation
    if (_print):
        print(cons_add_lhs_final, symbol, cons_add_rhs_final)
        
    # If user doesnt set up any penalty, Automatically add one
    if (Penalty == None):
        
        Penalty = np.max(np.abs(C))* 10   
        
    QUBO, dictionary = QUBO_Cons_selection(QUBO_pre, new_Constraint, new_Symbol, New_rhs, round_num, Penalty, dictionary)
    
    
    return QUBO, dictionary

In [3]:
def solve_master_qubo(QUBO_obj, cons_dictionary, round_num = None, Penalty = None, _print = False, dictionary = None):
    #symbol set initialization
    symbol_set = ["<=",">="] 
    
    # OBJ
    QUBO = QUBO_obj

    # count how many constraints
    constraint_number = len(cons_dictionary)
    
    # apply Constraint 1 by 1
    
    for index in range(constraint_number):
        #data preparation
        new_Constraint = cons_dictionary[index][0]
        new_Symbol = cons_dictionary[index][1]
        New_rhs = cons_dictionary[index][2]
    
        if New_rhs!= 0:
            symbol_index = symbol_set.index(new_Symbol)
            #print("input constraint:", index, "th constraint:", new_Constraint, symbol_set[symbol_index], New_rhs)
            symbol = (symbol_set[::int(np.sign(New_rhs))])[symbol_index]

            #if np.sign(New_rhs) == -1:
            #    symbol = ">="

            #  np.sign(New_rhs) == -1 ==>  switch >= and <= to the other symbol
            #  np.sign(New_rhs) == 1 ==>  remain >= and <= 


            cons_add_lhs_final = np.sign(New_rhs) * new_Constraint
            cons_add_rhs_final = np.sign(New_rhs) * New_rhs
        
        else:       
            cons_add_lhs_final = new_Constraint
            cons_add_rhs_final = New_rhs
            symbol = new_Symbol
        

        # print out the lasted the equation
        if (_print):
            print("input constraint:", index, "th constraint:", cons_add_lhs_final, symbol, cons_add_rhs_final)

        # If user doesnt set up any penalty, Automatically add one
        if (Penalty == None):

            Penalty = Penalty = np.max(np.abs(QUBO_obj))* 10    
        
        QUBO, dictionary = QUBO_Cons_selection(QUBO, cons_add_lhs_final, symbol, cons_add_rhs_final, round_num, Penalty)
    
    return QUBO, dictionary

In [8]:
def solve_for_value(QUBO, answer_length):
    
    # OBJ

    # Select a solver
    sampler = LeapHybridSampler()

    sampleset = sampler.sample_qubo(QUBO)
    
    OP = sampleset.to_pandas_dataframe()
    
    OP_final = OP.to_numpy()[0]
    
    #print(OP_final , answer_length)
    result = OP_final[:answer_length]
    
        
    return result, sampleset.info

In [ ]:
def cqm_solve_for_value(cqm, answer_length):
    
    # OBJ

    # Select a solver
    sampler = LeapHybridCQMSampler()    

    sampleset = sampler.sample_cqm(cqm, label="SDK Examples - Solar Panel Strategy")
    
    best = sampleset.filter(lambda row: row.is_feasible).first
    
    result = np.zeros(answer_length)
    
    for key, val in best.sample.items():
        
        index = int(key.replace("x_",""))
        
        if index < answer_length:
            
            result[index] = val 
    
    return result, sampleset.info

In [14]:
def spot_search(QUBO_obj, contraint_dict, answer_length):
    
    print("spot searching ......")
    

    QUBO, _ = solve_master_qubo(QUBO_obj, contraint_dict)
        
    # OBJ

    x, time_info = solve_for_value(QUBO, answer_length)
      
    value_x = x 
        
    print("Searching the spot done, Now return the result")

    return x, time_info

In [4]:
def answer_analyse(result, general_obj, len_of_t, general_t_coeff):
    
    #print(result, general_obj,general_t_coeff)
    
    value = dot(general_obj, result)
    
    t = dot(general_t_coeff ,result[:len_of_t])
    
    x = result[len_of_t:]
    
    return (t, x, value)
    

In [ ]:
def cqm_spot_search(QUBO_obj, contraint_dict, answer_length):
    
    print("cqm spot searching ......")
    
    x = [dimod.Binary(f'x_{i}') for i in range(len(QUBO_obj))]
    
    cqm = dimod.ConstrainedQuadraticModel()
    
    cqm.set_objective(sum(sum(QUBO_obj[i][j] * x[i] * x[j] for i in range(len(QUBO_obj))) for j in range(len(QUBO_obj)))) 
    
    for order in range(len(contraint_dict)):
        
        lhs = contraint_dict[order][0]
        
        rhs = contraint_dict[order][2]
    
        cqm.add_constraint( sum(lhs[i] * x[i] for i in range(len(QUBO_obj))) <= rhs, label=f'constraint_{order}')
    
    # OBJ

    x, time_info = cqm_solve_for_value(cqm, answer_length)
      
    value_x = x 
        
    print("Searching the spot done, Now return the result")

    return x, time_info